In [1]:
import sys
from unittest.mock import patch
from urllib3.util.retry import Retry

# Patch la classe Retry pour accepter method_whitelist
original_init = Retry.__init__

def patched_init(self, *args, **kwargs):
    if 'method_whitelist' in kwargs:
        kwargs['allowed_methods'] = kwargs.pop('method_whitelist')
    return original_init(self, *args, **kwargs)

Retry.__init__ = patched_init

# Maintenant on peut importer pytrends
from pytrends.request import TrendReq
import pandas as pd
import time
import re
from tqdm import tqdm
from pathlib import Path
import os
from datetime import datetime

In [ ]:
# ========= PHASE A : COLLECTE BRUTE (SANS FILTRAGE) =========

# ---- 0) TES SEEDS ----
seed_terms = [
    "moroccan food", "restaurant in morocco",
    "morocco hotels", "marrakech hotels", "agadir hotels", "tangier hotels", 
    "chefchaouen hotels", "riad marrakech", "riad fes",
    "flights to morocco", "royal air maroc", "marrakech flights", 
    "casablanca flights", "agadir flights", "tangier flights", "fes flights", "morocco map",
    "morocco travel", "visit morocco", "morocco tourism", "things to do in morocco",
    "sahara desert tour", "merzouga tour", "atlas mountains tour", 
    "essaouira day trip", "ouzoud waterfalls", "ait benhaddou", "places in morocco",
    "shopping in morocco", "marrakech souk", "hammam morocco", "argan oil morocco",
    "morocco weather", "marrakech weather", "agadir weather", 
    "climate of morocco", "best time to visit morocco",
    "morocco visa", "morocco itinerary", "morocco safety"
]

# ---- 1) Paramètres optimisés ----
GEOS = ["", "US", "GB", "FR", "ES", "DE"]
CATS = [0, 67]
TIMEFRAMES = ["today 12-m", "today 5-y", "2020-01-01 2024-12-31"]
TOP_K = 25
SLEEP = 1.5
MAX_DEPTH = 2

pytrends = TrendReq(hl="en-US", tz=0, timeout=(10, 25), retries=2, backoff_factor=0.5)

def related_union(term):
    """Union des Related (Top+Rising) sur (geo, cat, timeframe)."""
    out = set()
    success_count = 0
    
    for tf in TIMEFRAMES:
        for geo in GEOS:
            for cat in CATS:
                try:
                    pytrends.build_payload([term], geo=geo, cat=cat, timeframe=tf, gprop="")
                    time.sleep(SLEEP)
                    
                    rq = pytrends.related_queries()
                    d = rq.get(term, {})
                    
                    for kind in ("top", "rising"):
                        df = d.get(kind)
                        if df is not None and len(df) > 0:
                            success_count += 1
                            qs = df["query"].astype(str).head(TOP_K).tolist()
                            cleaned = [re.sub(r"\s+", " ", q.lower().strip()) for q in qs if q.strip()]
                            out.update(cleaned)
                    
                except Exception as e:
                    if "429" not in str(e) and "quota" not in str(e).lower():
                        print(f"\n  ⚠ [{term[:30]}]: {str(e)[:80]}")
                    time.sleep(SLEEP * 2)
                    continue
    
    if success_count > 0:
        print(f"  ✓ {term[:40]:40} → {len(out)} related")
    else:
        print(f"  ✗ {term[:40]:40} → 0 related")
    
    return list(out)

# ---- 2) Depth 1
print("\n" + "="*70)
print("DEPTH 1: Collecting related queries from seed terms")
print("="*70 + "\n")

frontier = list(dict.fromkeys([t.strip() for t in seed_terms]))
all_terms = set(frontier)
depth1_new = []

for term in tqdm(frontier, desc="Depth 1"):
    rel = related_union(term)
    for r in rel:
        if r and r not in all_terms:
            all_terms.add(r)
            depth1_new.append(r)

# ---- 3) Depth 2
print("\n" + "="*70)
print(f"DEPTH 2: Collecting from {len(depth1_new)} new terms")
print("="*70 + "\n")

frontier = depth1_new[:]
depth2_new = []

if len(frontier) > 0:
    for term in tqdm(frontier, desc="Depth 2"):
        rel = related_union(term)
        for r in rel:
            if r and r not in all_terms:
                all_terms.add(r)
                depth2_new.append(r)
else:
    print("⚠️  No new terms from Depth 1, skipping Depth 2")

# ---- 4) Résultats
print("\n" + "="*70)
print("RESULTS")
print("="*70)
print(f"Seeds: {len(seed_terms)}")
print(f"Depth1 new terms: {len(depth1_new)}")
print(f"Depth2 new terms: {len(depth2_new)}")
print(f"Total RAW unique terms: {len(all_terms)}")

# ---- 5) Sauvegardes
pd.Series(depth1_new if depth1_new else [""], name="term").to_csv("keywords_depth1_raw.csv", index=False)
pd.Series(depth2_new if depth2_new else [""], name="term").to_csv("keywords_depth2_raw.csv", index=False)
pd.Series(sorted(all_terms), name="term").to_csv("keywords_all_raw.csv", index=False)

print("\n✓ Saved: keywords_depth1_raw.csv, keywords_depth2_raw.csv, keywords_all_raw.csv")
print("="*70 + "\n")


DEPTH 1: Collecting related queries from seed terms



Depth 1:   2%|▎         | 1/40 [01:31<59:38, 91.76s/it]

  ✓ moroccan food                            → 42 related


Depth 1:   5%|▌         | 2/40 [03:08<1:00:01, 94.78s/it]

  ✗ restaurant in morocco                    → 0 related


...

  ✗ is it safe to travel to morocco right no → 0 related


Depth 2:  65%|██████▍   | 386/595 [17:31:31<9:53:40, 170.43s/it] 

  ✗ costco travel                            → 0 related


Depth 2:  65%|██████▍   | 386/595 [17:32:59<9:30:08, 163.68s/it]


KeyboardInterrupt: 

In [22]:
pd.Series(depth1_new if depth1_new else [""], name="term").to_csv("keywords_depth1_raw.csv", index=False)


In [23]:
pd.Series(depth2_new if depth2_new else [""], name="term").to_csv("keywords_depth2_raw.csv", index=False)

In [24]:
pd.Series(sorted(all_terms), name="term").to_csv("keywords_all_raw.csv", index=False)


In [2]:
data=pd.read_csv("keywords_all_raw.csv")
data

,term
0,1 dollar to moroccan dirham
1,1 euro to moroccan dirham
2,1 moroccan dirham to usd
3,1 week morocco itinerary
4,10 day morocco itinerary
...,...
1873,wydad casablanca
1874,yves saint laurent
1875,yves saint laurent marrakech
1876,zawina morocco


In [5]:
# ========= PHASE B : FILTRAGE SÉMANTIQUE =========


RAW = Path("keywords_all_raw.csv")
assert RAW.exists(), f"{RAW} manquant."

# 1) Charger
raw = pd.read_csv(RAW)
raw.columns = [c.strip().lower() for c in raw.columns]
col = "term" if "term" in raw.columns else raw.columns[0]
terms = raw[col].astype(str).fillna("").str.strip()
terms = terms[terms != ""].drop_duplicates().tolist()

# 2) Normalisation + clé canonique
STOP = {"in", "to", "of", "the", "a", "for", "and", "&", "is", "are", "with"}

def normalize(t: str) -> str:
    t = t.lower()
    t = re.sub(r"\s+", " ", t).strip()
    return t

def canonical(t: str) -> str:
    toks = normalize(t).split()
    toks = [w for w in toks if w not in STOP]
    return " ".join(sorted(toks))

# 3) Patterns de BRUIT ÉVIDENT (pas de contenu touristique)
NOISE_PATTERNS = [
    r'\b(facebook|instagram|tiktok|youtube|reddit|twitter|whatsapp|pinterest)\b',
    r'\b(download|mp3|gif|wallpaper|apk|pdf free)\b',
    r'\b(live score|match|football score|betting)\b',
    r'\b(news today|breaking news|latest news)\b',
    r'\b(porn|xxx|sex|adult)\b',
    r'\b(login|sign in|account|password)\b',
    r'\b(cracked|hack|cheat|keygen)\b',
]

# 4) Patterns TOURISTIQUES POSITIFS (large spectre)
TOURISM_PATTERNS = [
    # Intentions de voyage
    r'\b(travel|trip|tour|visit|holiday|vacation|journey|explore|discover)\b',
    r'\b(things to do|places to visit|what to see|activities|attractions)\b',
    r'\b(itinerary|guide|tips|advice|plan|planning)\b',
    
    # Infrastructure & services
    r'\b(hotel|riad|hostel|resort|accommodation|stay|lodging|booking)\b',
    r'\b(flight|airline|airport|ticket|ferry|train|bus|taxi|transfer|car rental)\b',
    r'\b(restaurant|food|cuisine|dining|eat|breakfast|lunch|dinner)\b',
    
    # Expériences & culture
    r'\b(desert|beach|mountain|medina|souk|market|bazaar|hammam|spa)\b',
    r'\b(museum|palace|fort|kasbah|mosque|garden|park)\b',
    r'\b(culture|tradition|festival|music|art|handicraft)\b',
    r'\b(hiking|trekking|surfing|camel|quad|adventure|safari)\b',
    
    # Éléments pratiques
    r'\b(weather|climate|temperature|season|best time)\b',
    r'\b(visa|passport|entry|requirements|customs)\b',
    r'\b(price|cost|budget|cheap|luxury|expensive)\b',
    r'\b(map|location|distance|direction|how to get)\b',
    r'\b(safe|safety|security|danger|scam|tips)\b',
    r'\b(photo|photography|instagram|scenic|view|sunset)\b',
    
    # Destinations (Maroc & général)
    r'\b(morocco|moroccan|marrakech|casablanca|fez|fes|tangier|tanger)\b',
    r'\b(chefchaouen|essaouira|agadir|merzouga|merzuga|ouarzazate)\b',
    r'\b(sahara|atlas|rif|dunes|gorge|valley|oasis)\b',
    r'\b(ait benhaddou|todra|todgha|dakhla|taghazout|ifrane)\b',
    r'\b(rabat|meknes|tetouan|asilah|ourika|imlil)\b',
]

def is_noise(term: str) -> bool:
    """Détecte le bruit évident"""
    return any(re.search(pattern, term) for pattern in NOISE_PATTERNS)

def is_tourism_related(term: str) -> bool:
    """Détecte le contenu touristique par patterns sémantiques"""
    return any(re.search(pattern, term) for pattern in TOURISM_PATTERNS)

# 5) Filtrage avec audit
rows = []
seen_canon = set()
kept = []

for t in terms:
    t_norm = normalize(t)
    t_can = canonical(t_norm)
    reason = None
    status = "kept"
    
    # Étape 1: Éliminer le bruit évident
    if is_noise(t_norm):
        status, reason = "removed", "noise/platform/irrelevant"
    
    # Étape 2: Vérifier la pertinence touristique
    elif not is_tourism_related(t_norm):
        status, reason = "removed", "not_tourism_related"
    
    # Étape 3: Déduplication canonique
    elif t_can in seen_canon:
        status, reason = "removed", "duplicate_canonical"
    
    else:
        seen_canon.add(t_can)
        kept.append(t_norm)
    
    rows.append({
        "term_raw": t,
        "term_norm": t_norm,
        "canonical": t_can,
        "status": status,
        "reason": reason
    })

audit = pd.DataFrame(rows)

# 6) Export
Path("phase_b_out").mkdir(parents=True, exist_ok=True)
pd.Series(sorted(set(kept)), name="term").to_csv("phase_b_out/keywords_clean.csv", index=False)
audit.to_csv("phase_b_out/audit_filter.csv", index=False)

# 7) Statistiques détaillées
print(f"\n{'='*60}")
print(f"RÉSULTATS DU FILTRAGE SÉMANTIQUE")
print(f"{'='*60}")
print(f"Input total:        {len(terms):,}")
print(f"Gardés:             {len(set(kept)):,}")
print(f"Supprimés:          {len(audit[audit.status=='removed']):,}")
print(f"\nRaisons de suppression:")
for reason, count in audit[audit.status=='removed']['reason'].value_counts().items():
    print(f"  - {reason}: {count:,}")
print(f"\n✓ Fichiers sauvegardés dans phase_b_out/")
print(f"  • keywords_clean1.csv")
print(f"  • audit_filter.csv")


RÉSULTATS DU FILTRAGE SÉMANTIQUE
Input total:        1,878
Gardés:             882
Supprimés:          996

Raisons de suppression:
  - not_tourism_related: 888
  - duplicate_canonical: 105
  - noise/platform/irrelevant: 3

✓ Fichiers sauvegardés dans phase_b_out/
  • keywords_clean1.csv
  • audit_filter.csv


a suprumé 

In [11]:
df1=pd.read_csv("keywords_clean.csv")
df2=pd.read_csv("phase_b_out/keywords_clean.csv")
L=list(df1["term"])
T=list(df2["term"])

In [25]:
s=set(L)
t=set(T)
len(t)

882

In [27]:
tout=s.union(t)


In [ ]:
dat=pd.read_csv("./dataset_final_avec_lags.csv")
a=list(dat.columns)
a.remove("TES")

82

In [68]:
dat

,TES,zawina morocco,morocco travel restrictions,mamounia hotel marrakech,is morocco safe,marrakech morocco,agadir morocco,marrakech hotels,morocco travel requirements,morocco trips,...,morocco vs france,marrakech weather december,usa vs morocco,rayan morocco,morocco world cup,argentina vs morocco,morocco earthquake death toll,morocco vs portugal,morocco portugal,spain morocco
0,444408.0,0.0,0.0,43.0,9.0,72.0,70.0,63.0,5.0,73.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,494208.0,0.0,23.0,43.0,16.0,18.0,15.0,60.0,40.0,22.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,624848.0,0.0,100.0,60.0,13.0,27.0,28.0,69.0,23.0,32.0,...,1.0,0.0,3.0,0.0,1.0,1.0,0.0,1.0,1.0,2.0
3,327017.0,0.0,9.0,34.0,24.0,70.0,81.0,52.0,10.0,65.0,...,0.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
4,382407.0,0.0,0.0,37.0,22.0,73.0,84.0,50.0,10.0,79.0,...,0.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,574847.0,0.0,0.0,50.0,44.0,55.0,40.0,43.0,5.0,50.0,...,0.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
156,3718.0,74.0,0.0,79.0,64.0,64.0,57.0,59.0,4.0,71.0,...,0.0,100.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
157,594992.0,0.0,0.0,37.0,76.0,66.0,49.0,44.0,5.0,64.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
158,636217.0,0.0,0.0,47.0,6.0,65.0,50.0,41.0,17.0,72.0,...,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,1.0


In [ ]:
a=set(a)
results=tout - a
results=list(results)
dict={"term":results}
data=pd.DataFrame(dict)
data.to_csv("keys.csv")


In [65]:
data[data["term"]=="royal air maroc airline"]

,term
745,royal air maroc airline


In [67]:
data[745:].to_csv("keys3.csv",index=False)

suprumé

In [48]:
# ========= PHASE C : COLLECTE TRENDS POUR MOTS CLE =========

# ===== Paramètres =====
KEYWORDS_CSV = "keywords_clean.csv"
OUT_CSV      = "trends_raw_monthly_new.csv"
CHECKPOINT   = "trends_checkpoint.csv"  # sauvegarde intermédiaire
FAILED_CSV   = "failed_keywords.csv"
TIMEFRAME    = "2010-01-01 2025-06-30"
GEO          = ""
CAT          = 0
SLEEP_SEC    = 3.0                      # AUGMENTÉ: pause entre chunks
MAX_RETRIES  = 2                        # RÉDUIT pour éviter ban
BACKOFF      = 3.0                      # pause plus longue
RESET_EVERY  = 8                        # reset tous les X chunks (réduit)
PAUSE_ON_429 = 120                      # pause de 2 minutes si 429
# =====================

# 1) Charger les mots-clés
kw = results
kw = list(kw)
print(f"📋 Keywords à collecter: {len(kw)}")

# 2) Fonction pour créer TrendReq avec protection
def create_pytrends():
    try:
        return TrendReq(hl="en-US", tz=0, timeout=(20,40), retries=1, backoff_factor=1.0)
    except Exception as e:
        print(f"⚠️  Erreur création instance: {e}")
        time.sleep(10)
        return None

# 3) Charger checkpoint si existe
def load_checkpoint():
    if os.path.exists(CHECKPOINT):
        df = pd.read_csv(CHECKPOINT, index_col=0, parse_dates=True)
        collected = list(df.columns)
        print(f"🔄 Checkpoint trouvé: {len(collected)} termes déjà collectés")
        return df, collected
    return pd.DataFrame(), []

# 4) Fonction de collecte ULTRA-DEFENSIVE
def fetch_trends_monthly(terms, timeframe=TIMEFRAME, geo=GEO, cat=CAT, sleep=SLEEP_SEC):
    """Collecte avec système de checkpoint et gestion 429."""
    
    # Charger progrès existant
    existing_df, already_collected = load_checkpoint()
    
    # Filtrer termes déjà collectés
    terms_to_fetch = [t for t in terms if t not in already_collected]
    if not terms_to_fetch:
        print("✅ Tous les termes déjà collectés!")
        return existing_df
    
    print(f"📥 Reste à collecter: {len(terms_to_fetch)} termes")
    
    chunks = [terms_to_fetch[i:i+5] for i in range(0, len(terms_to_fetch), 5)]
    frames = [existing_df] if not existing_df.empty else []
    failed_chunks = []
    
    # Instance initiale
    py = create_pytrends()
    if py is None:
        print("❌ Impossible de créer l'instance TrendReq. Vérifiez votre connexion.")
        return existing_df if not existing_df.empty else pd.DataFrame()
    
    consecutive_429 = 0  # compteur d'erreurs 429 consécutives
    
    for idx, group in enumerate(tqdm(chunks, desc="Downloading Google Trends")):
        success = False
        
        # Reset périodique
        if idx > 0 and idx % RESET_EVERY == 0:
            print(f"\n🔄 Reset connexion (chunk {idx}/{len(chunks)})")
            time.sleep(sleep * 2)
            py = create_pytrends()
            if py is None:
                print("⚠️  Reset échoué, continue avec ancienne instance")
        
        # Essais avec abandon rapide
        for attempt in range(MAX_RETRIES):
            try:
                py.build_payload(group, timeframe=timeframe, geo=geo, cat=cat, gprop="")
                df = py.interest_over_time()
                
                if df is not None and not df.empty and len(df) > 0:
                    df = df.drop(columns=["isPartial"], errors="ignore")
                    
                    if not df.index.empty:
                        df = df.resample("MS").mean()
                        frames.append(df)
                        success = True
                        consecutive_429 = 0  # reset compteur
                        
                        # 🔥 SAUVEGARDER IMMÉDIATEMENT
                        if len(frames) > 1:
                            temp = pd.concat(frames, axis=1)
                            temp = temp.T.groupby(level=0).max().T
                            temp.to_csv(CHECKPOINT, index=True)
                        
                        break
                else:
                    time.sleep(sleep)
                    
            except Exception as e:
                error_msg = str(e).lower()
                
                # Détection 429
                if "429" in error_msg or "quota" in error_msg or "rate" in error_msg:
                    consecutive_429 += 1
                    wait_time = PAUSE_ON_429 * consecutive_429  # augmente progressivement
                    
                    print(f"\n🚨 RATE LIMIT #{consecutive_429} détecté!")
                    print(f"⏸️  PAUSE FORCÉE: {wait_time}s ({wait_time/60:.1f} min)")
                    print(f"⏰ Reprise à {datetime.now().strftime('%H:%M:%S')}")
                    
                    # Si trop de 429, arrêt total
                    if consecutive_429 >= 3:
                        print("\n⛔ ARRÊT: Trop d'erreurs 429 consécutives")
                        print("💡 Solution: Attendre 1-2 heures ou changer d'IP")
                        failed_chunks.extend(chunks[idx:])  # marquer le reste comme échoué
                        break
                    
                    time.sleep(wait_time)
                    py = create_pytrends()  # nouvelle instance
                    if py is None:
                        break
                    
                elif "timeout" in error_msg or "connection" in error_msg:
                    wait_time = sleep * (attempt + 2)
                    print(f"⚠️  Timeout. Attente {wait_time:.1f}s...")
                    time.sleep(wait_time)
                    
                    if attempt == MAX_RETRIES - 1:
                        py = create_pytrends()
                else:
                    if attempt == MAX_RETRIES - 1:
                        print(f"❌ Échec chunk {idx+1}: {type(e).__name__}")
                        failed_chunks.append(group)
                        py = create_pytrends()
                    time.sleep(sleep * (attempt + 1))
        
        # Vérifier si on doit arrêter
        if consecutive_429 >= 3:
            break
        
        if success:
            time.sleep(sleep)
        else:
            time.sleep(sleep * 3)  # pause plus longue après échec
    
    # Rapport final
    if failed_chunks:
        print(f"\n⚠️  {len(failed_chunks)} chunk(s) échoué(s)")
        failed_terms = [t for chunk in failed_chunks for t in chunk]
        pd.DataFrame({"term": failed_terms}).to_csv(FAILED_CSV, index=False)
        print(f"💾 Termes échoués -> {FAILED_CSV}")
    
    if not frames:
        return pd.DataFrame()
    
    # Consolidation finale
    out = pd.concat(frames, axis=1)
    out = out.T.groupby(level=0).max().T
    
    try:
        start = pd.to_datetime(TIMEFRAME.split()[0])
        end = pd.to_datetime(TIMEFRAME.split()[-1])
        out = out.loc[start:end]
    except:
        pass
    
    return out

# 5) Exécution
print("\n🚀 Début de la collecte...")
print("⚠️  Le script peut prendre plusieurs heures avec les pauses anti-ban\n")

trends = fetch_trends_monthly(kw)

if not trends.empty:
    print(f"\n📊 Shape finale: {trends.shape}")
    print(f"✅ {trends.shape[1]}/{len(kw)} termes collectés ({trends.shape[1]/len(kw)*100:.1f}%)")
    
    trends.to_csv(OUT_CSV, index=True)
    print(f"💾 Sauvegardé -> {OUT_CSV}")
    
    # Nettoyer checkpoint
    if os.path.exists(CHECKPOINT):
        os.remove(CHECKPOINT)
        print("🧹 Checkpoint nettoyé")
else:
    print("\n❌ Aucune donnée collectée")
    print("💡 Le checkpoint contient peut-être des données partielles")

print("\n" + "="*60)
print("📌 CONSEILS SI BLOQUÉ:")
print("1. Attendre 1-2 heures avant de relancer")
print("2. Relancer ce script (il reprendra automatiquement)")
print("3. Utiliser un VPN/proxy ou changer de réseau")
print("4. Réduire SLEEP_SEC à 5.0 ou plus")
print("="*60)

📋 Keywords à collecter: 801

🚀 Début de la collecte...
⚠️  Le script peut prendre plusieurs heures avec les pauses anti-ban

📥 Reste à collecter: 801 termes



🚨 RATE LIMIT #1 détecté!
⏸️  PAUSE FORCÉE: 120s (2.0 min)
⏰ Reprise à 17:27:53

🚨 RATE LIMIT #2 détecté!
⏸️  PAUSE FORCÉE: 240s (4.0 min)
⏰ Reprise à 17:29:54


KeyboardInterrupt: 

In [126]:
df1=pd.read_csv("trends_raw_monthly.csv")
df2=pd.read_csv("trends_raw_monthly1.csv")

In [ ]:
df2 = df2[~df2["date"].astype(str).str.contains("2020")]
df2 = df2[~df2["date"].astype(str).str.contains("2020")]

In [ ]:
df1.to_csv("trends_raw_monthly.csv", index=False)
df2.to_csv("trends_raw_monthly1.csv",index=False)

In [137]:
df2

,date,1 week morocco itinerary,5 star marrakech hotels,agadir hotels,agadir morocco,agadir weather,agadir weather february,agadir weather may,air morocco,argan oil morocco,...,riad marrakech pas cher,savoy le grand hotel marrakech,spain morocco,tangier flights,tangier morocco,things to do in morocco,usa vs morocco,visit morocco,where is morocco,zawina morocco
0,2010-01-01,0.0,55.0,87.0,95.0,61.0,32.0,0.0,78.0,0.0,...,93.0,0.0,1.0,50.0,44.0,36.0,0.0,22.0,8.0,0.0
1,2010-02-01,0.0,49.0,72.0,87.0,73.0,20.0,0.0,69.0,0.0,...,87.0,0.0,1.0,45.0,45.0,29.0,0.0,23.0,9.0,0.0
2,2010-03-01,0.0,72.0,79.0,84.0,57.0,0.0,53.0,69.0,5.0,...,100.0,0.0,1.0,48.0,48.0,39.0,0.0,23.0,8.0,0.0
3,2010-04-01,0.0,60.0,77.0,80.0,40.0,0.0,0.0,82.0,4.0,...,88.0,0.0,1.0,76.0,48.0,34.0,0.0,21.0,7.0,0.0
4,2010-05-01,0.0,0.0,72.0,71.0,36.0,0.0,0.0,71.0,5.0,...,76.0,0.0,1.0,74.0,49.0,40.0,0.0,22.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,2025-02-01,0.0,42.0,37.0,90.0,94.0,32.0,72.0,53.0,76.0,...,12.0,71.0,1.0,89.0,86.0,80.0,1.0,83.0,23.0,86.0
182,2025-03-01,0.0,37.0,33.0,84.0,97.0,4.0,79.0,50.0,78.0,...,11.0,78.0,1.0,79.0,77.0,80.0,1.0,82.0,21.0,100.0
183,2025-04-01,0.0,30.0,29.0,77.0,65.0,0.0,94.0,52.0,80.0,...,9.0,85.0,1.0,89.0,94.0,73.0,1.0,77.0,20.0,85.0
184,2025-05-01,0.0,27.0,30.0,76.0,48.0,8.0,18.0,53.0,82.0,...,6.0,85.0,1.0,87.0,93.0,74.0,1.0,75.0,19.0,82.0
